In [0]:
%run "/Workspace/Users/sundaramloganathan@outlook.com/includes/storage_acc_config"

In [0]:
%run "../includes/folder_path"

In [0]:
from pyspark.sql.types import IntegerType, StringType, StructField, StructType
from pyspark.sql.functions import col, lit, to_timestamp

name_schema = StructType([StructField('forename', StringType(), True),
                             StructField('surname', StringType(), True)])


In [0]:
driver_schema = StructType([StructField('driverId', IntegerType(), False),
                                 StructField('driverRef', StringType(), True),
                                 StructField('code', StringType(), True),
                                 StructField('dob', StringType(), True),
                                 StructField('name', name_schema , True),
                                 StructField('nationality', StringType(), True),
                                 StructField('number', IntegerType(), True),
                                 StructField('url', StringType(), True)])

In [0]:
driver_df = spark.read.options(header=True).schema(driver_schema).json(f"{bronze_folder_path}/drivers.json")

In [0]:
from pyspark.sql.functions import current_timestamp, concat, lit, col, to_timestamp, when, regexp_replace

driver_upd_df = driver_df.withColumnRenamed("driverId", "driver_id") \
                         .withColumnRenamed("driverRef", "driver_ref") \
                         .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                         .withColumn("ingestion_date", current_timestamp()) \
                         .drop("url") 

In [0]:
driver_upd_df.write.mode("overwrite").parquet(f"{silver_folder_path}/drivers")